In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)


In [2]:
def generate_deals(n=500):
    segments = ["SMB", "MM", "ENT"]

    rows = []
    for i in range(n):
        segment = np.random.choice(segments, p=[0.4, 0.35, 0.25])
        num_products = np.random.choice([1, 2, 3], p=[0.5, 0.35, 0.15])
        contract_years = np.random.choice([1, 2, 3], p=[0.5, 0.3, 0.2])
        deal_type = np.random.choice(["New", "Expansion"], p=[0.6, 0.4])

        base_price = {"SMB": 15000, "MM": 60000, "ENT": 180000}[segment]
        list_price = base_price * num_products * contract_years

        rows.append({
            "deal_id": i,
            "segment": segment,
            "num_products": num_products,
            "contract_years": contract_years,
            "deal_type": deal_type,
            "list_price": list_price
        })

    return pd.DataFrame(rows)

deals = generate_deals()
deals.head()


,deal_id,segment,num_products,contract_years,deal_type,list_price
0,0,SMB,3,2,New,90000
1,1,SMB,1,1,Expansion,15000
2,2,MM,2,1,Expansion,120000
3,3,ENT,1,1,New,180000
4,4,SMB,2,1,New,30000


In [3]:
def recommend_discount(segment, num_products, contract_years, deal_type):
    base = {"SMB": 0.25, "MM": 0.18, "ENT": 0.12}[segment]
    bundle_adj = -0.03 if num_products >= 2 else 0
    term_adj = -0.02 if contract_years >= 3 else 0
    expansion_adj = -0.02 if deal_type == "Expansion" else 0

    return max(base + bundle_adj + term_adj + expansion_adj, 0.05)

deals["recommended_discount"] = deals.apply(
    lambda r: recommend_discount(
        r.segment, r.num_products, r.contract_years, r.deal_type
    ),
    axis=1
)


In [ ]:
deals["actual_discount"] = (
    deals["recommended_discount"]
    + np.random.normal(0, 0.03, len(deals))
).clip(0.05, 0.4)

deals["final_price"] = deals["list_price"] * (1 - deals["actual_discount"])
